In [1]:
import requests
import json
import openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
import calendar
import datetime
from statistics import mean
import sqlite3
from numpy import arange
import pandas as pd
import time
import re

import config

In [2]:
###Database connection
conn = sqlite3.connect('GeoPVWattData.db')
c = conn.cursor()

In [4]:
# US Cities with population > 1000
# floc = "simplemaps_uscities_basicv1.6\\uscities.csv"
# df_all = pd.read_csv(floc)
# df = df_all[df_all['population']>1000]
# df=df.sort_values(by='population', ascending=False)
# latvals = list(df['lat'])
# lonvals = list(df['lng'])

# World cities (outside the us) 
# floc = "simplemaps_worldcities_basicv1.6\\worldcities.csv"
# df_all = pd.read_csv(floc)
# df = df_all[df_all['country']!='United States']
# df=df.sort_values(by='population', ascending=False)
# latvals = list(df['lat'])
# lonvals = list(df['lng'])
# countries = list(df['country'])

# #Europe
# floc = "simplemaps_worldcities_basicv1.6\\worldcities.csv"
# df_all = pd.read_csv(floc)
# df = df_all[df_all['lat']>30]
# df = df_all[(df_all['country']!='United States') & (df_all['lat']>30) & (df_all['lat']<70) & 
#             (df_all['lng']>-30) & (df_all['lng']<60)]
# df=df.sort_values(by='population', ascending=False)
# df=df.sort_values(by='country', ascending=False)
# latvals = list(df['lat'])
# lonvals = list(df['lng'])
# countries = list(df['country'])
# print(countries)

# Outside Europe (picking some specific ranges outside the edges of Europe)
floc = "simplemaps_worldcities_basicv1.6\\worldcities.csv"
df_all = pd.read_csv(floc)
df = df_all[df_all['lat']>30]

df = df_all[(df_all['country']!='United States') & (((df_all['lat']>15) & (df_all['lat']<30) & (df_all['lng']>-45) & (df_all['lng']<75))
                                                   | ((df_all['lat']>70) & (df_all['lat']<85) & (df_all['lng']>-45) & (df_all['lng']<75))
                                                   | ((df_all['lat']>15) & (df_all['lat']<85) & (df_all['lng']>-45) & (df_all['lng']<-30))
                                                   | ((df_all['lat']>15) & (df_all['lat']<85) & (df_all['lng']>60) & (df_all['lng']<75)))]



# df = df_all[(df_all['country']!='United States') & ((df_all['lat']>15) & (df_all['lat']<85)) & ((df_all['lat']<30) | (df_all['lat']>70))
#            & ((df_all['lng']>-45) & (df_all['lng']<75)) & ((df_all['lng']<-30) | (df_all['lng']>60))]
# df = df_all[(df_all['country']!='United States') & (df_all['lat']>15) & (df_all['lat']<85) & 
#             (df_all['lng']>-45) & (df_all['lng']<60)]
# df = df_all[(df_all['country']=='Greenland') & (df_all['lat']>15) & (df_all['lat']<85) & 
#             (df_all['lng']>-45) & (df_all['lng']<75)]
# df = df_all[(df_all['country']=='Greenland') & (df_all['lat']>30) & (df_all['lat']<70) & 
#             (df_all['lng']>-30) & (df_all['lng']<60)]
# df_all = df
# df = df_all[(df_all['country']=='Greenland') & ((df_all['lat']<30) | (df_all['lat']>70)) & 
#             ((df_all['lng']<-30) | (df_all['lng']>60))]


# df = df_all[(df_all['country']=='Greenland') & 
#             ((df_all['lng']<-30) | (df_all['lng']>60))]

# df1 = df_all[(df_all['country']=='Greenland') & (df_all['lat']>15) & (df_all['lat']<85) & 
#             (df_all['lng']>-45) & (df_all['lng']<75)]
# df2 = df_all[(df_all['country']=='Greenland') & (df_all['lat']>30) & (df_all['lat']<70) & 
#             (df_all['lng']>-30) & (df_all['lng']<60)]
# common = df1.merge(df2,on=['lat','lng','population','country'])
# df = df1[~df1.isin(df2)].dropna()

df=df.sort_values(by='population', ascending=False)
df=df.sort_values(by='country', ascending=False)
latvals = list(df['lat'])
lonvals = list(df['lng'])
countries = list(df['country'])
print(countries)
print(latvals)
print(lonvals)
print(len(countries))


['Yemen', 'Yemen', 'Yemen', 'Yemen', 'Yemen', 'Yemen', 'Yemen', 'Yemen', 'Yemen', 'Yemen', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'Uzbekistan', 'United Arab Emirates', 'United Arab Emirates', 'United Arab Emirates', 'United Arab Emirates', 'United Arab Emirates', 'United Arab Emirates', 'United Arab Emirates', 'United Arab Emirates', 'Turkmenistan', 'Turkmenistan', 'Turkmenistan', 'Turkmenistan', 'Tajikistan', 'Tajikistan', 'Tajikistan', 'Tajikistan', 'Tajikistan', 'Tajikistan', 'Tajikistan', 'Tajikistan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Sudan', 'Spain', 'Spain', 'Spain', 'Senegal', 'Senegal', 'Senegal', 'Sene

In [5]:
#Select latitude longitute range
# latvals = [80]     #Range -90 to 90
# latvals = arange(10,90,1)
# lonvals = [-180]     #Rangel -180 to 180
# lonvals = arange(-180,-50,1)

#Tilt angle (Range 0-90, 0 = horizontal, 90 = vertical, 20 = typical tracking array)
#tilt = 0
tiltvals = [0,90]
tiltvals = [0,30,90]

#Azimuth (Range 0-360, 0=N, 45=NE, 90=E, 135=SE, 180=S, 225=SW, 270=W, 315=NW)
#azimuth = 180
# azimuthvals = [0,45,90,135,180,225,270,315]
azimuthvals = [0,90,180,270]


In [6]:
def azimuthtitle(a):
    switcher = {
        0:'N',
        45:'NE', 
        90:'E', 
        135:'SE', 
        180:'S', 
        225:'SW', 
        270:'W', 
        315:'NW' 
    }
    return switcher.get(a,a)

In [7]:
 #loop through latitude
rc = 0
for x in range(len(latvals)):
    lat = latvals[x]
    lon = lonvals[x]
  
    #loop through tilts
    for i in range(len(tiltvals)):
        tilt = tiltvals[i]

        #loop through angles
        for j in range(len(azimuthvals)):
            time.sleep(3.6)
            azimuth = azimuthvals[j]
            az = azimuthtitle(azimuth)

            ##########Query PVWatts to get solar data
            url = "https://developer.nrel.gov/api/pvwatts/v6.json"
            payload = "-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"\"\r\n\r\n\r\n-----011000010111000001101001--\r\n"
            headers = {'content-type': 'multipart/form-data; boundary=---011000010111000001101001'}
            #####US DATA
#             querystring = {"api_key":config.api_key,"system_capacity":"4",
#                            "module_type":"0","losses":"10","array_type":"0","tilt":tilt,"azimuth":azimuth,
#                            "lat":lat,"dataset":"tmy3","lon":lon}
            #####International Data
            querystring = {"api_key":config.api_key,"system_capacity":"4",
                           "module_type":"0","losses":"10","array_type":"0","tilt":tilt,"azimuth":azimuth,
                           "lat":lat,"dataset":"INTL","lon":lon,"radius":"0"}

            response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
#             print(response.text)

            #####Break out of loop if no data exists
            er  = json.loads(response.text)['errors']
            if len(er) != 0:
                print(er)
                break
            ###########Solar data and used data parsing from output of query

            #parse out the latitude, longitude, and city location used for analysis
            used_lat = json.loads(response.text)['station_info']['lat']
            used_lon = json.loads(response.text)['station_info']['lon']
            used_city = json.loads(response.text)['station_info']['city']
            used_city = re.sub('"', '',used_city)
            used_city = re.sub("'", '',used_city)
            used_state = json.loads(response.text)['station_info']['state']
            used_country = countries[x]
            used_country = re.sub('"', '',used_country)
            used_country = re.sub("'", '',used_country)
#             used_country = 'United States'

            sm = json.loads(response.text)['outputs']['solrad_monthly']
            sa = json.loads(response.text)['outputs']['solrad_annual']
            sm_ave = round(mean(sm),3) 
            sm_minus = round(mean(sm)-min(sm),3)
            sm_plus = round(max(sm)-mean(sm),3)  

            sql = f"""INSERT INTO LatLonData(lat, lon, city, state, tilt,azimuth, azimuthname, 
                    solrad_jan, solrad_feb, solrad_mar, solrad_apr, solrad_may, solrad_jun,
                    solrad_jul, solrad_aug, solrad_sep, solrad_oct, solrad_nov, solrad_dec,
                    solrad_average, solrad_minus, solrad_plus, solrad_annual, country) 
                    VALUES (
                    {used_lat:.4f}, {used_lon:.4f}, '{used_city}', '{used_state}', {tilt}, {azimuth}, 
                    '{az}', {sm[0]:.3f}, {sm[1]:.3f}, {sm[2]:.3f}, {sm[3]:.3f}, {sm[4]:.3f}, 
                    {sm[5]:.3f}, {sm[6]:.3f}, {sm[7]:.3f}, {sm[8]:.3f}, {sm[9]:.3f}, {sm[10]:.3f},
                    {sm[11]:.3f}, {sm_ave}, {sm_minus}, {sm_plus}, {sa:.3f}, '{used_country}'
                    )"""
#             print(sql)
            print(x)
            try:
#                 print(response.text)
                c.execute(sql)
                conn.commit()
                rc += 1
                if rc % 100 == 0:
                    print(response.text)
                    print(c.fetchall())
            except sqlite3.IntegrityError:
                pass
            if tilt == 0:
                break

KeyboardInterrupt: 

In [34]:
sql = "SELECT * FROM LatLonData"
c.execute(sql)
print(c.fetchall())

[(37.617, -122.4, '"SAN FRANCISCO INTL AP"', 'CA', 0.0, 0.0, 'N', 2.138, 3.18, 4.407, 5.675, 6.424, 7.153, 7.124, 6.377, 5.468, 3.764, 2.531, 2.131, 4.698, 2.567, 2.455, 4.698), (37.617, -122.4, '"SAN FRANCISCO INTL AP"', 'CA', 90.0, 0.0, 'N', 0.636, 0.865, 0.984, 1.319, 1.488, 1.752, 1.843, 1.514, 1.152, 0.987, 0.718, 0.625, 1.157, 0.531, 0.686, 1.157), (37.617, -122.4, '"SAN FRANCISCO INTL AP"', 'CA', 90.0, 45.0, 'NE', 0.69, 1.021, 1.372, 2.05, 2.34, 2.726, 2.652, 2.318, 1.647, 1.254, 0.809, 0.668, 1.629, 0.961, 1.097, 1.629), (37.617, -122.4, '"SAN FRANCISCO INTL AP"', 'CA', 90.0, 90.0, 'E', 1.352, 1.906, 2.375, 3.01, 3.158, 3.474, 3.422, 3.307, 2.77, 2.278, 1.678, 1.473, 2.517, 1.165, 0.957, 2.517), (37.617, -122.4, '"SAN FRANCISCO INTL AP"', 'CA', 90.0, 135.0, 'SE', 2.401, 3.064, 3.225, 3.288, 3.062, 3.098, 3.251, 3.519, 3.645, 3.332, 2.902, 2.855, 3.137, 0.736, 0.508, 3.137), (37.617, -122.4, '"SAN FRANCISCO INTL AP"', 'CA', 90.0, 180.0, 'S', 3.096, 3.894, 3.694, 3.077, 2.496, 2.

In [14]:
conn.close()